In [1]:
from skimage.feature import graycomatrix
from skimage.feature import graycoprops

In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.express as px
import plotly

from bread.vis import *
from bread.data import Features, SegmentationFile, Microscopy
from bread.algo.lineage import LineageGuesser, LineageGuesserNN, LineageGuesserML, LineageGuesserNearestCell, accuracy
from bread.data import Lineage

In [3]:
seg = SegmentationFile.from_h5(
    '../../data/segmentations/colony003_segmentation.h5').get_segmentation("FOV0")
microscopy = Microscopy.from_tiff(
    '../../data/microscopies/colony003_microscopy.tif')
feat = Features(seg, nn_threshold=12, scale_length=1, scale_time=1, microscopy=microscopy)

In [12]:
# feat.cell_mean_intensity(1,1)
seg.cell_ids(154)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61]

In [ ]:
feat.cell_texture(2,1)

{'contrast': 2440.883758710085,
 'energy': 0.16274341764097258,
 'correlation': 0.4914472200237478}

In [ ]:
feat.cell_texture(1,2)

{'contrast': 1653.0935051240167,
 'energy': 0.11075640512605349,
 'correlation': 0.49912984704676977}

In [ ]:
feat.cell_texture(2,2)

{'contrast': 2892.2107423596844,
 'energy': 0.1227982393912742,
 'correlation': 0.37970996443706606}

In [ ]:
feat.cell_r_equiv(1,1)
feat.cell_r_equiv(1,2)
feat.cell_r_equiv(2,1)
feat.cell_r_equiv(2,2)



17.283918998391353

In [ ]:
def cell_texture( time_id: int, cell_id: int, distance: int = 1) -> float:
    fov = 'FOV0'
    assert microscopy is not None, "Microscopy object not set"
    cell_mask = segmentation.data[time_id] == cell_id
    image = microscopy.get_frame(fov, time_id)*cell_mask
    cm = feat._cm(cell_id, time_id)
    r = feat.cell_r_equiv(time_id, cell_id)
    # crop the image around the cell
    xmin = int(cm[1]-r)
    xmax = int(cm[1]+r)
    ymin = int(cm[0]-r)
    ymax = int(cm[0]+r)
    cell_image = image[ymin:ymax, xmin:xmax]
    distance = [1,3]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    cell_image_uint8 = ((cell_image - np.min(cell_image)) / (np.max(cell_image) - np.min(cell_image)) * 255).astype(np.uint8)

    glcm = graycomatrix(cell_image_uint8, distance, angles, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast').mean()
    texture_features = {}
    texture_features['contrast'] = contrast

    energy = graycoprops(glcm, 'energy').mean()
    texture_features['energy'] = energy

    correlation = graycoprops(glcm, 'correlation').mean()
    texture_features['correlation'] = correlation
    return correlation , contrast, energy


In [ ]:
segmentation = seg

cell_texture(1,1)

<function skimage.feature.texture.graycoprops(P, prop='contrast')>